<a href="https://colab.research.google.com/github/dbizzaro/Minesweeper/blob/main/Minesweeper_with_SATsolver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install python-sat[pblib,aiger]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from pysat.card import CardEnc
from pysat.solvers import Minisat22
import numpy as np
import pandas as pd
from itertools import combinations
from scipy.special import binom

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
class Board:
  
  def __init__(self, n_rows, n_cols, n_mines, seed=None):
    self.n_rows = n_rows
    self.n_cols = n_cols
    self.n_mines = n_mines
    self.board = np.full((n_rows, n_cols), -1, dtype='short')
    self.__mines = set()
    np.random.seed(seed)
    placed = 0
    while (placed < n_mines):
      row = np.random.randint(0,n_rows)
      col = np.random.randint(0,n_cols)
      if ((row,col) not in self.__mines):
        self.__mines.add((row,col))
        placed += 1

  def adjacent_cells(self, position, only_among_still_covered=True):
    adj = []
    for x in range(position[0]-1, position[0]+2):
        for y in range(position[1]-1, position[1]+2):
            if (x>=0 and y>=0 and x<self.n_rows and y<self.n_cols and (x, y) != position):
              if ((not only_among_still_covered) or self.board[x,y]<0):
                adj.append((x,y))
    return adj

  def n_adjacent_mines(self, position):
    adj = self.adjacent_cells(position)
    n = 0
    for pos in adj:
      if (pos in self.__mines):
        n += 1
    return n

  def randomly_fill_in(self, n_observed, seed=None):
    observed = 0
    while (observed < n_observed):
      row = np.random.randint(0,self.n_rows)
      col = np.random.randint(0,self.n_cols)
      if (self.board[row,col] == -1 and ((row,col) not in self.__mines)):
        self.board[row,col] = self.n_adjacent_mines((row, col))
        observed += 1
  
  def show(self, display_mines=False):
    df = pd.DataFrame(self.board, dtype='object')
    df = df.replace(-1, '').replace(-2, '🚩')
    if display_mines:
      for pos in self.__mines:
        df.iloc[pos] = '💣'
    display(df)

  def still_covered_cells(self, separate_cells_with_an_uncovered_neighbor=False):
    covered_cells_with_an_uncovered_neighbor = []
    covered_cells_without_an_uncovered_neighbor = []
    covered_cells = []
    for row in range(self.n_rows):
      for col in range(self.n_cols):
        if (self.board[row,col] < 0):
          if not separate_cells_with_an_uncovered_neighbor:
            covered_cells.append((row,col))
          elif len(self.adjacent_cells((row, col), True))!=len(self.adjacent_cells((row, col), False)):
            covered_cells_with_an_uncovered_neighbor.append((row, col))
          else:
            covered_cells_without_an_uncovered_neighbor.append((row, col))
    if not separate_cells_with_an_uncovered_neighbor:
      return covered_cells
    return covered_cells_with_an_uncovered_neighbor, covered_cells_without_an_uncovered_neighbor

  def reveal_position(self, position, show_board=False):
    n_adj_mines = self.n_adjacent_mines(position)
    if (position in self.__mines):
      print("BOOM! Exploded mine in position {}.". format(position))
      self.show(True)
      return -2
    if (self.board[position] < 0):
      self.board[position] = n_adj_mines
      print("Revealed position {}, which has {} neighboring mines.".format(position, n_adj_mines))
      if show_board:
        self.show()
      return n_adj_mines
    print('Invalid position')
    return -3


In [ ]:
def variable_mine_at(n_cols, position):
    return 1 + position[0] * n_cols + position[1]

In [ ]:
def proposition_exactly_k_neighboring_mines(board, position, k):
  adj = board.adjacent_cells(position)
  n = len(adj)
  clauses=[]
  for I in combinations(adj, n-k+1):  # at least k
    clauses.append([variable_mine_at(board.n_cols, i) for i in I])
  for I in combinations(adj, k+1):  # at most k
    clauses.append([-variable_mine_at(board.n_cols, i) for i in I])
  #print(clauses)
  return clauses

maximum number clauses from single cardinality constrain: $\binom{8}{5}+ \binom{8}{3}=112$

In [ ]:
def proposition_total_number_of_mines(board):
  available_positions = board.still_covered_cells()
  variables = [variable_mine_at(board.n_cols, pos) for pos in available_positions]
  n = board.n_rows * board.n_cols
  k = board.n_mines
  cnf = CardEnc.equals(variables, k, top_id=n)
  #print(cnf.clauses)
  return cnf.clauses

In [ ]:
def generate_propositions(board, satsolver):
  for row in range(board.n_rows):
    for col in range(board.n_cols):
      k = board.board[row,col]
      if k >=0:
        satsolver.append_formula(proposition_exactly_k_neighboring_mines(board, (row,col), k))
        satsolver.add_clause([-variable_mine_at(board.n_cols, (row,col))])
  #print(satsolver.nof_vars())
  #print(satsolver.nof_clauses())
  satsolver.append_formula(proposition_total_number_of_mines(board))
  #print(satsolver.nof_vars())
  #print(satsolver.nof_clauses())

In [ ]:
def check_safe_position_at(board, position, satsolver): # if UNSAT having a mine in that position (i.e. if VALID not having a mine in that position, i.e. SAFE position)
  return not satsolver.solve(assumptions=[variable_mine_at(board.n_cols, position)]) 

def check_mine_at(board, position, satsolver): # if UNSAT not having a mine in that position (i.e. if VALID having a mine in that position)
  return not satsolver.solve(assumptions=[-variable_mine_at(board.n_cols, position)])

def count_models(satsolver, assumptions):
  return sum(1 for _ in satsolver.enum_models(assumptions=assumptions))
  #lista=list(satsolver.enum_models(assumptions=assumptions))
  #print(lista)
  #return len(lista)

In [ ]:
def most_probable_safe_position(board, satsolver):

  top_id = satsolver.nof_vars()

  def assumption_exactly_k_mines_among_free_cells(k):
    assumption = [variable_mine_at(board.n_cols, free_cells[i]) for i in range(k)]
    assumption += [-variable_mine_at(board.n_cols, free_cells[i+k]) for i in range(n_free_cells-k)]
    assumption += [-i for i in range(top_id+1, satsolver.nof_vars()+1)] + [satsolver.nof_vars()+1]
    #print(satsolver.nof_vars())
    return assumption

  #def assumption_exactly_k_mines_among_free_cells2():
    #assumption = [variable_mine_at(board.n_cols, free_cells[i]) for i in range(k)]
    #assumption += [-variable_mine_at(board.n_cols, free_cells[i+k]) for i in range(n_free_cells-k)]
    #assumption = [-i for i in range(top_id+1, satsolver.nof_vars()+1)] + [satsolver.nof_vars()+1]
    #print(satsolver.nof_vars())
    #return assumption

  bounded_cells, free_cells = board.still_covered_cells(separate_cells_with_an_uncovered_neighbor=True)
  n_bounded_cells = len(bounded_cells)
  n_free_cells = len(free_cells)
  counter_n_models = 0
  for n_mines_in_free_cells in range(max(0,board.n_mines-n_bounded_cells), min(n_free_cells, board.n_mines)):
    assumption = assumption_exactly_k_mines_among_free_cells(n_mines_in_free_cells)
    counter_n_models += count_models(satsolver, assumption) * binom(n_free_cells-1, n_mines_in_free_cells)
    #print("binomial", binom(n_free_cells-1, n_mines_in_free_cells), n_free_cells-1, n_mines_in_free_cells, counter_n_models)
  #print("counter_first", counter_n_models)
  #print("classical", count_models(satsolver, assumption_exactly_k_mines_among_free_cells2()+[-variable_mine_at(board.n_cols, free_cells[0])]))
  if n_free_cells > 0: 
    maximum = counter_n_models
    best_option = free_cells[0]
    print("maximum_first", maximum)
  else:
    maximum = 0 
    best_option = None
  for position in bounded_cells:
    counter_n_models = 0
    for n_mines_in_free_cells in range(max(0,board.n_mines-n_bounded_cells+1), min(n_free_cells, board.n_mines)+1):
      assumption = assumption_exactly_k_mines_among_free_cells(n_mines_in_free_cells) + [-variable_mine_at(board.n_cols, position)]
      counter_n_models += count_models(satsolver, assumption) * binom(n_free_cells, n_mines_in_free_cells)
      #print("models", count_models(satsolver, assumption))
      #print("binomial", binom(n_free_cells, n_mines_in_free_cells), n_free_cells, n_mines_in_free_cells, counter_n_models)
    #print("counter_final", counter_n_models)
    #print("classical", count_models(satsolver, assumption_exactly_k_mines_among_free_cells2()+[-variable_mine_at(board.n_cols, position)]))
    if counter_n_models > maximum:
      maximum = counter_n_models
      print("maximum", maximum)
      best_option = position
  satsolver.append_formula([[-i] for i in range(top_id+1, satsolver.nof_vars()+1)])
  return best_option


In [ ]:
def play_game(board, steps_after_which_showing_board=10):
  counter_for_showing_board = 0
  n_mines_found = 0
  solver = Minisat22()
  generate_propositions(board, solver)
  game_over_flag = False
  while not game_over_flag:
    safe_positions_found = 0
    bounded_cells, free_cells = board.still_covered_cells(separate_cells_with_an_uncovered_neighbor=True)
    if len(free_cells) > 0:
      cells_to_try = bounded_cells + [free_cells[0]]
    else:
      cells_to_try = bounded_cells
    for position in cells_to_try:
      if (board.board[position] == -1 and check_mine_at(board, position, solver)): 
        solver.add_clause([variable_mine_at(board.n_cols, position)]) # add clause saying we've found a mine
        board.board[position] = -2
        n_mines_found += 1
        print("Found mine in position {}".format(position))
      if (board.board[position] == -1 and check_safe_position_at(board, position, solver)):  # if position safe, then ...
        n_adj_mines = board.reveal_position(position, True) # ... reveal position ...
        if n_adj_mines < 0:
          game_over_flag = True
          print("Game Over: the program has lost")
          break
        solver.append_formula(proposition_exactly_k_neighboring_mines(board, position, n_adj_mines))
        solver.add_clause([-variable_mine_at(board.n_cols, position)])
        safe_positions_found += 1
    if safe_positions_found == 0 and not game_over_flag and n_mines_found == board.n_mines:
      print("The program has won!")
      board.show(True)
      break
    elif safe_positions_found == 0 and not game_over_flag:
      position_to_try = most_probable_safe_position(board, solver)
      n_adj_mines = board.reveal_position(position_to_try, True) # ... reveal position ...
      if n_adj_mines < 0:
        game_over_flag = True
        print("Game Over: the program has lost")
        break
      solver.append_formula(proposition_exactly_k_neighboring_mines(board, position_to_try, n_adj_mines))
      solver.add_clause([-variable_mine_at(board.n_cols, position_to_try)])
  solver.delete()
  return not game_over_flag

In [ ]:
def statistics(n_rows, n_cols, n_mines, sample_size):
  for i in range(sample_size):
    new_board = Board(n_rows, n_cols, n_mines)
    play_game(new_board)

In [ ]:
#example = Board(20, 30, 120)
#example.randomly_fill_in(0)
#example.show(True)

In [ ]:
#play_game(example)

TO DO:


*   verbosity
*   comments
*   statistics

